# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [37]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [38]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [40]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Modeling Apache Cassandra Tables and Queries. 

### Data to be used for modeling will come from the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [41]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [42]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [43]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## session_items table

#### Create session_items table. This table will store information about songs and artists that were played during sessions. Primary keys used are session_id and item_in_session as these attributes will be used in the where clauses for queries.

Attributes Defined:
- session_id 
- item_in_session
- artist
- song_title
- song

In [44]:
create_table_query1 = "CREATE TABLE IF NOT EXISTS session_items \
                        (session_id int, item_in_session int, artist text, song_title text, song_length float, \
                        PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(create_table_query1)
except Exception as e:
    print(e)

#### Read in the data from csv and insert the respective columns into the created session_items table. String formats are changed to adhere to the table data types

In [45]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_items(session_id, item_in_session, artist, song_title, song_length) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Perform the select query to retrieve artist, song_title and song_length for session_id 338 and item_in_session 4.

In [46]:
query1 = "SELECT artist, song_title, song_length FROM session_items WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## user_session table

#### Create user_session table. This table will store information about users and the songs they played in each session. Primary keys used are (user_id and session_id) as partition keys and item_in_session as clustering column.

Attributes Defined:
 - user_id
 - session_id
 - item_in_session
 - first_name
 - last_name
 - artist
 - song_title

In [47]:
create_table_query2 = "CREATE TABLE IF NOT EXISTS user_session \
                        (user_id int, session_id int, item_in_session int, first_name text, last_name text, \
                        artist text, song_title text, \
                        PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(create_table_query2)
except Exception as e:
    print(e)

#### Read in the data from csv and insert the respective columns into the created user_session table. String formats are changed to adhere to the table data types

In [48]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session(user_id, session_id, item_in_session, first_name, last_name, artist, song_title) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

#### Perform the select query to retrieve artist, song_title and user name for user_id 10 and session_id 182.

In [49]:
query2 = "SELECT artist, song_title, first_name, last_name FROM user_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## songs_listened table

#### Create songs_listened table. This table will store information about songs played by users. Primary keys used is song_title (partition key) and user_id (clustering column).

Attributes Defined:
- song_title
- user_id
- first_name
- last_name

In [50]:
create_table_query3 = "CREATE TABLE IF NOT EXISTS songs_listened \
                        (song_title text, user_id int, first_name text, last_name text, \
                        PRIMARY KEY(song_title, user_id))"
try:
    session.execute(create_table_query3)
except Exception as e:
    print(e)

#### Read in the data from csv and insert the respective columns into the created user_session table. String formats are changed to adhere to the table data types

In [51]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_listened(song_title, user_id, first_name, last_name) "
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Perform the select query to retrieve user names who listened to song_title 'All Hands Against His Own'.

In [52]:
query3 = "SELECT first_name, last_name FROM songs_listened WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Drop the tables before closing out the sessions

In [53]:
query = "drop table session_items"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_listened"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Close the session and cluster connection¶

In [54]:
session.shutdown()
cluster.shutdown()